<a href="https://colab.research.google.com/github/hcarlos0877/MNIST/blob/main/MNIST_DIGITS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyspark

In [167]:
# Read data from .csv in PySpark
import pyspark as ps
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split

spark = (
    SparkSession
    .builder # драйвер
    .master("local") #local - отправляется на драйвер
    .appName("spark2mnist")
    .getOrCreate()
)

img_rows, img_cols, img_depth = 28, 28, 1
num_classes = 10


path = '/content/drive/MyDrive/MNIST/data/digit-recognizer/train.csv'

mnist_df = (
    spark.read
    .option("header", "true")
    .option('delimiter',',')
    .csv(path))

def spark_convert(mnist_df):
  """ convert spark df to X and y numpy"""
  y_col = mnist_df.columns[0]
  # y_temp = mnist_df.withColumn(y_col, mnist_df[0].cast(IntegerType())).select(y_col)
  y_temp = mnist_df.withColumn(y_col, mnist_df[0].cast("int")).select(y_col)
  yy = y_temp.select('*').toPandas().to_numpy()
  yyy = keras.utils.to_categorical(yy, num_classes)
  # yy = mnist_df.withColumn(mnist_df.columns[0], mnist_df[0].cast(IntegerType())).select()
  x_temp = mnist_df.select(*(col(c).cast("int").alias(c) for c in mnist_df.columns[1:]))
  x_numpy = x_temp.select('*').toPandas().to_numpy()/255
  xx = x_numpy.reshape(x_temp.count(), img_rows, img_cols, img_depth)
  # xx = np.reshape(x_temp, (x_temp.count(), img_rows, img_cols, img_depth) , order='C')
  print("X shape is ",(x_temp.count(), len(x_temp.columns)))
  return xx, yyy

# mnist_df.select([col for col in mnist_df.columns]).show(5)

xx, yy = spark_convert(mnist_df)

X shape is  (42000, 784)


In [146]:
xx.shape

(42000, 28, 28, 1)

In [147]:
yy.shape

(42000, 1)

In [130]:
print("Shape is ",(x_temp.count(), len(x_temp.columns)))

Shape is  (42000, 784)


In [136]:
x_numpy.shape

(42000, 784)

In [139]:
yy.show(3)

+-----+
|label|
+-----+
|    1|
|    0|
|    1|
+-----+
only showing top 3 rows



In [109]:
x_temp.count()

42000

In [97]:
x_temp.count()

42000

In [95]:
len(x_temp.columns)

784

In [105]:
yy.show(5)

+-----+
|label|
+-----+
|    1|
|    0|
|    1|
|    4|
|    0|
+-----+
only showing top 5 rows



In [ ]:
mnist_df.toPandas()

In [ ]:
# convert data to 

In [ ]:
mnist_df.columns[0]
mnist_df.columns[1:]

In [ ]:
mnist_df.printSchema()

In [ ]:
mnist_df.show(5)

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras

img_rows, img_cols = 28, 28
num_classes = 10

def prep_data(raw):
    y = raw[:, 0]
    out_y = keras.utils.to_categorical(y, num_classes)
    
    x = raw[:,1:]
    num_images = raw.shape[0]
    out_x = x.reshape(num_images, img_rows, img_cols, 1)
    out_x = out_x / 255
    return out_x, out_y



digits_file = "/content/drive/MyDrive/MNIST/data/digit-recognizer/train.csv"
digits_data = np.loadtxt(digits_file, skiprows=1, delimiter=',')
X, y = prep_data(digits_data)



In [3]:

X_train, X_val, Y_train, Y_val = train_test_split(X, y, test_size = 0.1, random_state=42)


In [4]:
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, Flatten
from tensorflow.keras.layers import BatchNormalization
from keras.models import Sequential  # Model type to be used
from keras.layers.core import Dense, Dropout, Activation # Types of layers to be used in our model


model = Sequential() 
# Convolution Layer 1
model.add(Conv2D(32, (3, 3), input_shape=(28,28,1))) # 32 different 3x3 kernels -- so 32 feature maps
model.add(BatchNormalization(axis=-1))               # normalize each feature map before activation
convLayer01 = Activation('relu')                     # activation
model.add(convLayer01)

# Convolution Layer 2
model.add(Conv2D(32, (3, 3)))                        # 32 different 3x3 kernels -- so 32 feature maps
model.add(BatchNormalization(axis=-1))               # normalize each feature map before activation
model.add(Activation('relu'))                        # activation
convLayer02 = MaxPooling2D(pool_size=(2,2))          # Pool the max values over a 2x2 kernel
model.add(convLayer02)

# Convolution Layer 3
model.add(Conv2D(64,(3, 3)))                         # 64 different 3x3 kernels -- so 64 feature maps
model.add(BatchNormalization(axis=-1))               # normalize each feature map before activation
convLayer03 = Activation('relu')                     # activation
model.add(convLayer03)

# Convolution Layer 4
model.add(Conv2D(64, (3, 3)))                        # 64 different 3x3 kernels -- so 64 feature maps
model.add(BatchNormalization(axis=-1))               # normalize each feature map before activation
model.add(Activation('relu'))                        # activation
convLayer04 = MaxPooling2D(pool_size=(2,2))          # Pool the max values over a 2x2 kernel
model.add(convLayer04)
model.add(Flatten())                                 # Flatten final 4x4x64 output matrix into a 1024-length vector

# Fully Connected Layer 5
model.add(Dense(512))                                # 512 FCN nodes
model.add(BatchNormalization())                      # normalization
model.add(Activation('relu'))                        # activation

# Fully Connected Layer 6                       
model.add(Dropout(0.2))                              # 20% dropout of randomly selected nodes
model.add(Dense(10))                                 # final 10 FCN nodes
model.add(Activation('softmax'))                     # softmax activation

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 batch_normalization (BatchN  (None, 26, 26, 32)       128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 24, 24, 32)       128       
 hNormalization)                                                 
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0

In [5]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [6]:
# data augmentation prevents overfitting by slightly changing the data randomly
# Keras has a great built-in feature to do automatic augmentation

gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                         height_shift_range=0.08, zoom_range=0.08)

# We can then feed our augmented data in batches
# Besides loss function considerations as before, this method actually results in significant memory savings
# because we are actually LOADING the data into the network in batches before processing each batch

# Before the data was all loaded into memory, but then processed in batches.

train_generator = gen.flow(X_train, Y_train, batch_size=64)

In [7]:
# We can now train our model which is fed data by our batch loader
# Steps per epoch should always be total size of the set divided by the batch size

# SIGNIFICANT MEMORY SAVINGS (important for larger, deeper networks)
# one-hot format classes
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import warnings

nb_classes = 10 # number of unique digits
batch_size = 64
epochs = 20

model.fit_generator(train_generator, steps_per_epoch=X_train.shape[0] // batch_size, epochs = epochs, verbose=1, 
                    validation_data = (X_val,Y_val), validation_steps=X_val.shape[0] // batch_size)

<ipython-input-7-e3bbcf50ebc0>:14: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch=X_train.shape[0] // batch_size, epochs = epochs, verbose=1,


Epoch 1/20
590/590 [==============================] - 138s 231ms/step - loss: 0.1535 - accuracy: 0.9516 - val_loss: 0.0603 - val_accuracy: 0.9805
Epoch 2/20
444/590 [=====================>........] - ETA: 32s - loss: 0.0641 - accuracy: 0.9792

KeyboardInterrupt: ignored

In [ ]:
score = model.evaluate(X_val, Y_val)
print('Val accuracy:', score[1])

In [ ]:
# Predict the values from the validation dataset
Y_pred = model.predict([X[0]])
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred, axis = 1)

In [195]:
Y_pred_classes

array([1, 0, 1])